In [1]:
from transformers import pipeline

In [2]:
s_pipeline = pipeline("sentiment-analysis", model = "distilbert-base-uncased-finetuned-sst-2-english")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


In [18]:
import pandas as pd

In [19]:
df = pd.read_csv('/content/starbucks_reviews_LA.csv')

In [33]:
df.head()

,Review
0,I normally buy my Grande Pike at this Starbuck...
1,Very bad customer service. The person who take...
2,"Starbucks retail locations in Hollywood, Calif..."
3,Today is 05/29/2016. I would like to make a co...
4,I enjoy my coffee everyday. Unfortunately it h...


In [39]:
def sentiment(df):
  for index,row in df.iterrows():
        review_text = row['Review']
        sentiment_result = s_pipeline(review_text)
        print(f"Review {index + 1}: {sentiment_result}")

In [40]:
sentiment(df)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Review 1: [{'label': 'NEGATIVE', 'score': 0.9991408586502075}]
Review 2: [{'label': 'NEGATIVE', 'score': 0.9996446371078491}]
Review 3: [{'label': 'NEGATIVE', 'score': 0.9994818568229675}]
Review 4: [{'label': 'NEGATIVE', 'score': 0.9991256594657898}]
Review 5: [{'label': 'NEGATIVE', 'score': 0.9994786381721497}]
Review 6: [{'label': 'NEGATIVE', 'score': 0.9974105954170227}]
Review 7: [{'label': 'NEGATIVE', 'score': 0.993990421295166}]
Review 8: [{'label': 'NEGATIVE', 'score': 0.9994107484817505}]
Review 9: [{'label': 'NEGATIVE', 'score': 0.9976099729537964}]
Review 10: [{'label': 'NEGATIVE', 'score': 0.9998100399971008}]
Review 11: [{'label': 'NEGATIVE', 'score': 0.9995346069335938}]
Review 12: [{'label': 'NEGATIVE', 'score': 0.9446586966514587}]


In [41]:
df_1 = pd.read_csv('/content/starbucks_reviews_NY.csv')

In [42]:
sentiment(df_1)

Review 1: [{'label': 'NEGATIVE', 'score': 0.9995989203453064}]
Review 2: [{'label': 'NEGATIVE', 'score': 0.9983964562416077}]
Review 3: [{'label': 'NEGATIVE', 'score': 0.9996119141578674}]
Review 4: [{'label': 'POSITIVE', 'score': 0.9978042244911194}]
Review 5: [{'label': 'NEGATIVE', 'score': 0.9967027306556702}]
Review 6: [{'label': 'NEGATIVE', 'score': 0.9991859793663025}]
Review 7: [{'label': 'POSITIVE', 'score': 0.7781310677528381}]
Review 8: [{'label': 'POSITIVE', 'score': 0.9997243285179138}]
Review 9: [{'label': 'NEGATIVE', 'score': 0.9997050166130066}]
Review 10: [{'label': 'NEGATIVE', 'score': 0.9991369843482971}]
Review 11: [{'label': 'NEGATIVE', 'score': 0.999686598777771}]
Review 12: [{'label': 'NEGATIVE', 'score': 0.998715877532959}]
Review 13: [{'label': 'POSITIVE', 'score': 0.9988563060760498}]
Review 14: [{'label': 'NEGATIVE', 'score': 0.999777615070343}]


In [43]:
import nltk

In [44]:
from nltk import corpus

In [46]:
from nltk.corpus import stopwords

In [47]:
from nltk.stem import WordNetLemmatizer

In [50]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [52]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [63]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [61]:
def lematize_text(df):
  lem = WordNetLemmatizer()
  for index,row in df.iterrows():
    review_text = row['Review']
    tokens = nltk.word_tokenize(review_text.lower())
    lem_token = [lem.lemmatize(word) for word in tokens]
    lem_sentence = ' '.join(word for word in lem_token if word not in stopwords.words('english'))
    print({index+1},lem_sentence)

In [64]:
lematize_text(df)

{1} normally buy grande pike starbucks : 138 central ave , los angeles , ca 90012. today decided add bagel cream cheese complete breakfast . left go back asap cream cheese given wa opened used one . asked manager accommodated say piece chris served . told wa even thinking ? ? ? ? plague world would . apologized never affected n't know . asked give somebody else cream cheese ? ? ? contaminated . trust starbucks . training employee ? ? ? disgusting . wish could attach picture cream cheese . consumer check buy time .
{2} bad customer service . person take order doe smile rude . asked counter sandwich heated rudely responded `` '' stern blunt voice , apology whatsoever . also asked another person working bar water , said `` one minute plz '' ignored started walking around kitchen 3 minute . finally , went back order area order water , new barista wa polite addressed request . overall , rude staff , worst staff ever seen starbucks . outlet either . experience ha made never go starbucks , ev

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [57]:
def get_top_keywords(text,n=10):
  vectorizer = TfidfVectorizer(max_features = 100)
  matrix = vectorizer.fit_transform([text])
  feature_names = vectorizer.get_feature_names_out()
  return feature_names